In [ ]:
model_name = "EfficientNetB2_224_regression"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
import math
import cv2

from sklearn.metrics import cohen_kappa_score

import matplotlib.pyplot as plt

import gc
from tqdm import tqdm

import os
import scipy as sp
from functools import partial

from nn_generator import PreTrainDataGenerator, TrainDataGenerator, TestDataGenerator
from lib import plot_training

# basic
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Dropout, BatchNormalization
from keras.layers import Flatten, GlobalAveragePooling2D, GlobalMaxPool2D, Softmax
from keras.activations import elu
from keras import optimizers
from keras.utils import Sequence

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# model
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras_applications.densenet import DenseNet121, DenseNet169
from keras_applications.resnext import ResNeXt101
from keras_applications.nasnet import NASNetMobile

from efficientnet.keras import EfficientNetB5, EfficientNetB2

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", # specify GPU number
        allow_growth=True
    )
)
set_session(tf.Session(config=config))

In [ ]:
df_train_15 = pd.read_csv('../data/trainLabels15.csv')
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
df_train.head()

In [ ]:
pretrain_img_path = "../data/resized_train_15/"
img_size = 224

In [ ]:
def get_train_model(class_num=5, img_size=224, optimizer=optimizers.Adam(lr=1e-4)):
    inp = Input((img_size, img_size, 3))
    
    base_model = EfficientNetB2(include_top=False, weights="imagenet", input_tensor=inp)
    x = base_model(inp)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(5, activation=elu)(x)
    out = Dense(1, activation="linear")(x)
    
    model = Model(inputs=inp, outputs=out)
    model.compile(loss="mse", optimizer=optimizer, metrics=["acc"])
    return model

In [ ]:
# まずはじめに10 epochs学習する
pretrain_generator = PreTrainDataGenerator(df=df_train_15, batch_size=16, img_size=224,
                                           data_path=pretrain_img_path, augmentation=True, shuffle=True)
first_val_generator = TrainDataGenerator(df=df_train, batch_size=16)

model = get_train_model(img_size=img_size, class_num=5, optimizer=optimizers.Adam(lr=1e-4))

print("*** Model summary ***")
print(model.summary())
print("*********************")

history = model.fit_generator(pretrain_generator, epochs=10, validation_data=first_val_generator,
                              steps_per_epoch=pretrain_generator.__len__(), verbose=1)

plot_training(history)

In [ ]:
# 10 epochs以降はearly stoppingでval_loss見て止める
early_stopping = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", patience=1, factor=0.5, verbose=1)

history = model.fit_generator(pretrain_generator, epochs=150, validation_data=first_val_generator,
                              steps_per_epoch=pretrain_generator.__len__(),
                              callbacks=[early_stopping, reduce_lr], verbose=1)

plot_training(history)

In [ ]:
model.save('./save/pretrain/'+model_name+"_pretrain.h5")